In [7]:
#import useful moduels
import numpy as np                       ##Used for numerical computations 
import pandas as pd                      ##Used for reading the data
import seaborn as sns                    #Used for plotting with more features
import matplotlib.pyplot as plt          #Used for plotting 
from nltk.corpus import stopwords        ##This is used to plot the number of stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize        ##This is used to divide the overall text data to tokens and sentences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer   ##Using the vectorizer to convert values
import tqdm                        ##Used for measuring the time it takes to get the things done 
import re                           ##Standard library for reading and substituting the word expressions 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import nltk                                ##Used for the natural language processing tasks 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import missingno as msno                      ## Used to plot the missing values that are present in our data
from wordcloud import WordCloud               ##It is used to plot the frequency of the words which determines their size
import plotly.express as px               ##This library is used for interactive visualization 
from plotly import graph_objects as go    ##We also have to use this along with plotly to get interactive visualization
import warnings                       ##We are going to filer some warnings and remove them when we try to import the libraries
warnings.filterwarnings("ignore")

[nltk_data] Error loading punkt: <urlopen error [WinError 10054]
[nltk_data]     远程主机强迫关闭了一个现有的连接。>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     远程主机强迫关闭了一个现有的连接。>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054]
[nltk_data]     远程主机强迫关闭了一个现有的连接。>
[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10054]
[nltk_data]     远程主机强迫关闭了一个现有的连接。>


In [8]:
#load data
TRAIN_PATH = 'data/'
TEST_PATH = 'data/'
df_train = pd.read_csv(TRAIN_PATH + 'train.csv', low_memory = False)
df_test = pd.read_csv(TEST_PATH + 'test.csv', low_memory = False)
df_sample = pd.read_csv(TEST_PATH + 'sample_submission.csv', low_memory = False)

df_test.head()

,id,url_legal,excerpt
0,1,https://docs.google.com/document/d/1rt1D0xDDpJ...,That is what the vendor been called the new wo...


In [9]:
df_train['excerpt'].iloc[1]

'All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\r\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, "All right, Dolly, you may go."\r\n"Oh, mother!" Dolly cried, overwhelmed with sudden delight. "Really?\r\nOh, I am so glad! Are you sure you\'re willing?"\r\n"I\'ve persuaded myself to be willing, against my will," returned Mrs. Fayre, whimsically. "I confess I just hate to have you go, but I can\'t bear to deprive you of the pleasure trip. And, as you say, it would also keep Dotty at home, and so, altogether, I think I shall have to give in."\r\n"Oh, you angel mother! You blessed lady! How good you are!" And Dolly flew around the table and gave her mother a hug that nearly suffocated her.'

In [10]:
df_train['excerpt'].apply(lambda text: text.replace('\n', ' ')).iloc[1]
df_train['excerpt'] = df_train['excerpt'].apply(lambda text: text.replace('\n', ' '))
df_test['excerpt'] = df_test['excerpt'].apply(lambda text: text.replace('\n', ' '))
df_train['excerpt'].iloc[1]

'All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\r At last, however, when the meal was nearly over, she smiled at her little daughter, and said, "All right, Dolly, you may go."\r "Oh, mother!" Dolly cried, overwhelmed with sudden delight. "Really?\r Oh, I am so glad! Are you sure you\'re willing?"\r "I\'ve persuaded myself to be willing, against my will," returned Mrs. Fayre, whimsically. "I confess I just hate to have you go, but I can\'t bear to deprive you of the pleasure trip. And, as you say, it would also keep Dotty at home, and so, altogether, I think I shall have to give in."\r "Oh, you angel mother! You blessed lady! How good you are!" And Dolly flew around the table and gave her mother a hug that nearly suffocated her.'

In [11]:
#preprocess function
def preprocessing_function(df):
    """
    This function takes into consideration the dataframe and extracts the text.
    In addition, it makes modifications to the text and converts it to a simpler form
    for machine learning processing respectively."""
    
    text_list = []
    for text in tqdm(df['excerpt'].values):
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = nltk.word_tokenize(text)
        [word for word in text if not word in set(stopwords.words("english"))]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(text)
        text_list.append(text)
    text_list = pd.Series(text_list)
    text_list.column = ['Converted_text']
    return text_list

In [12]:
preprocessed_text = preprocessing_function(df_train)

100%|██████████| 2834/2834 [02:43<00:00, 17.28it/s]


In [13]:
preprocessed_text_test = preprocessing_function(df_test)

100%|██████████| 1/1 [00:00<00:00,  7.08it/s]


In [14]:
# define useful features
def get_useful_features(df, stop_words):
    """
    The function would take the dataframe and stopwords and then, convert the excerpts into different features
    such as the number of sentences, words and the lenght of the lemmas created along with the overall preprocessed
    essay length."""
    sentences = []
    num_of_words = []
    sent_length = []
    word_length = []
    lemma_length = []
    num_of_lemmas = []
    preprocessed_essay_length = []
    initial_text_length = []
    num_of_sentences = []
    text_shortage = []
    
    for text in tqdm(df['excerpt'].values):
        
        initial_length = len(text)
        initial_text_length.append(initial_length)
        num_sentences = len(sent_tokenize(text))
        num_of_sentences.append(num_sentences)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = word_tokenize(text)
        num_words = len(text) 
        num_of_words.append(num_words)
        sent_length.append(num_words/num_sentences)
        word_length.append(initial_length/num_words)
        text = [word for word in text if not word in stop_words]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        #print(text)
        num_lemmas = len(text)
        num_of_lemmas.append(num_lemmas)
        text = " ".join(text)
        #print(text)
        preprocessed_essay_length_value = len(text)
        preprocessed_essay_length.append(preprocessed_essay_length_value)
        #print(preprocessed_essay_length)
        #print(num_lemmas)
        lemma_length.append(preprocessed_essay_length_value/num_lemmas)
        
        text_shortage.append(preprocessed_essay_length_value/initial_length)
        
    final_df = pd.concat([pd.Series(sent_length), pd.Series(num_of_words),
                             pd.Series(word_length), pd.Series(lemma_length),
                             pd.Series(num_of_sentences), pd.Series(initial_text_length),
                             pd.Series(num_of_lemmas), pd.Series(preprocessed_essay_length),
                             pd.Series(text_shortage)], axis = 1)
    final_df.columns = ["sentence_length", "num_of_words", "word_length",
                           "lemma_length", "num_of_sentences",
                           "initial_text_length", "num_of_lemmas",
                           "preprocessed_essay_length", "text_shortage"]
    
    return final_df
        

In [15]:
final_df = get_useful_features(df_train, stop_words = set(stopwords.words("english")))

100%|██████████| 2834/2834 [00:03<00:00, 792.66it/s]


In [16]:
final_df_test = get_useful_features(df_test, stop_words = set(stopwords.words("english")))

100%|██████████| 1/1 [00:00<00:00, 499.38it/s]


In [17]:
final_df.head()

,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,16.454545,181,5.508287,6.494505,11,997,91,591,0.592778
1,11.466667,172,5.476744,6.482353,15,942,85,551,0.584926
2,15.636364,172,5.302326,6.273810,11,912,84,527,0.577851
3,33.400000,167,5.449102,6.095745,5,910,94,573,0.629670
4,30.200000,151,4.788079,5.581081,5,723,74,413,0.571231


In [19]:
def generate_more_features(df: pd.DataFrame):
    """
    This function would create a dataframe of different useful features
    that are important for machine learning predictions respectively.
    """
    commas = []
    semicolon = []
    exclamations = []
    questions = []
    quotes = []
    periods = []
    longest_word = []
    
    for i in range(len(df)):
        
        #word_len = []
        text = df['excerpt'].iloc[i]
        commas.append(text.count(","))
        semicolon.append(text.count(";"))
        exclamations.append(text.count("!"))
        questions.append(text.count("?"))
        quotes.append(text.count('"'))
        periods.append(text.count('.'))
        word_len = [len(w) for w in text.split(" ")]
        longest_word.append(np.max(word_len))
        
    df_with_features =pd.concat((pd.Series(commas), pd.Series(semicolon), pd.Series(exclamations),
                               pd.Series(questions), pd.Series(quotes), pd.Series(periods),
                                pd.Series(longest_word)), axis = 1)
    df_with_features.columns = ["num_of_commas", "num_of_semicolons", "num_of_explamations",
                                "num_of_questions", "num_of_quotes", "num_of_periods", 
                                "longest_word"]
                                
    return df_with_features

In [20]:
df_with_more_features = generate_more_features(df_train)

In [21]:
df_with_more_features_test = generate_more_features(df_test)

In [22]:
df_with_more_features.head()

,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word
0,14,0,0,0,0,11,11
1,24,0,5,2,12,10,12
2,17,2,1,0,10,11,14
3,23,2,0,0,0,5,13
4,13,10,0,0,0,5,12


In [23]:
any(df_with_more_features.isnull())

True

In [24]:
df_with_more_features.isnull()

,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
2829,False,False,False,False,False,False,False
2830,False,False,False,False,False,False,False
2831,False,False,False,False,False,False,False
2832,False,False,False,False,False,False,False


In [25]:
df_with_more_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2834 entries, 0 to 2833
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   num_of_commas        2834 non-null   int64
 1   num_of_semicolons    2834 non-null   int64
 2   num_of_explamations  2834 non-null   int64
 3   num_of_questions     2834 non-null   int64
 4   num_of_quotes        2834 non-null   int64
 5   num_of_periods       2834 non-null   int64
 6   longest_word         2834 non-null   int32
dtypes: int32(1), int64(6)
memory usage: 144.0 KB


In [26]:
df_with_more_features.head()

,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word
0,14,0,0,0,0,11,11
1,24,0,5,2,12,10,12
2,17,2,1,0,10,11,14
3,23,2,0,0,0,5,13
4,13,10,0,0,0,5,12


In [27]:
df_train['excerpt'].iloc[140]

'Little Sara Crewe never went in or out of the house without reading that door plate and reflecting upon it. By the time she was twelve, she had decided that all her trouble arose because, in the first place, she was not "Select," and in the second she was not a "Young Lady." When she was eight years old, she had been brought to Miss Minchin as a pupil, and left with her. Her papa had brought her all the way from India. Her mamma had died when she was a baby, and her papa had kept her with him as long as he could. And then, finding the hot climate was making her very delicate, he had brought her to England and left her with Miss Minchin, to be part of the Select Seminary for Young Ladies. Sara, who had always been a sharp little child, who remembered things, recollected hearing him say that he had not a relative in the world whom he knew of, and so he was obliged to place her at a boarding-school, and he had heard Miss Minchin\'s establishment spoken of very highly.'

In [28]:
df_train['excerpt'].iloc[143]

'"You really wish me to understand, Brady, that not you alone, but all the elder boys—day-pupils and boarders alike—desire of your own free-will to devote your next Saturday\'s half-holiday to conveying this poor man\'s plants from his house at Brickland to the Rookwood sale?"\r "Yes, sir, that\'s what we want to do."\r "H\'m! Well, the proposal does you credit, and you certainly might employ your time much worse than in carrying it out. I don\'t think it would be right for me to refuse your request. Mr. Anderson, I feel sure, will be ready to help and advise you, if necessary, but as the idea is your own I should like you, as far as possible, to carry it out by yourselves."\r "Thank you, sir!" said Jack, and withdrew.\r It was evening when this dialogue took place. The day-boys had departed in an irritable frame of mind, on account of various annoyances of which they had been the victims during the past two days. Bacon had been tripped up twice by a piece of string, Hughes had found h

In [29]:
df_for_plotting = pd.concat([df_with_more_features, df_train['target']], axis = 1)

In [30]:
def Count_Vectorizer(df: pd.DataFrame):
    vectorizer = CountVectorizer()
    vectorizer.fit(df['excerpt'])
    converted_vector = vectorizer.transform(df['excerpt'])
    return converted_vector

In [31]:
Count_Vectorizer(df_train)

<2834x26833 sparse matrix of type '<class 'numpy.int64'>'
	with 294295 stored elements in Compressed Sparse Row format>

In [32]:
from nltk.corpus import stopwords

In [33]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    """
    This function would convert some short letters into the forms that one 
    could easily understand respectively.
    """
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [23]:
def preprocessing_function(df):
    """
    In this function, we are going to be performing the basic preprocessing 
    which is needed before giving to different vectorizers. This ensures that
    we are getting the best output values respectively.
    """
    
    stopwords_new = set(stopwords.words('english'))
    text_list = []
    for i in tqdm(range(len(df['excerpt']))):
        text = df['excerpt'].iloc[i]
        text = decontracted(text)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = [word for word in text.split(' ') if not word in stopwords_new]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        text_list.append(' '.join(text))
    return text_list


In [34]:
df_train_preprocessed = preprocessing_function(df_train)

100%|██████████| 2834/2834 [02:52<00:00, 16.47it/s]


In [35]:
df_test_preprocessed = preprocessing_function(df_test)

100%|██████████| 1/1 [00:00<00:00,  8.90it/s]


In [36]:
preprocessed_text.head()

0    when the young people returned to the ballroom...
1    all through dinner time mr fayre wa somewhat s...
2    a roger had predicted the snow departed a quic...
3    and outside before the palace a great garden w...
4    once upon a time there were three bear who liv...
dtype: object

In [37]:
df_with_more_features.head()

,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word
0,14,0,0,0,0,11,11
1,24,0,5,2,12,10,12
2,17,2,1,0,10,11,14
3,23,2,0,0,0,5,13
4,13,10,0,0,0,5,12


In [38]:
df_for_plotting.head()

,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,target
0,14,0,0,0,0,11,11,-0.340259
1,24,0,5,2,12,10,12,-0.315372
2,17,2,1,0,10,11,14,-0.580118
3,23,2,0,0,0,5,13,-1.054013
4,13,10,0,0,0,5,12,0.247197


In [39]:
final_df.head()

,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,16.454545,181,5.508287,6.494505,11,997,91,591,0.592778
1,11.466667,172,5.476744,6.482353,15,942,85,551,0.584926
2,15.636364,172,5.302326,6.273810,11,912,84,527,0.577851
3,33.400000,167,5.449102,6.095745,5,910,94,573,0.629670
4,30.200000,151,4.788079,5.581081,5,723,74,413,0.571231


In [40]:
df_complete = pd.concat((preprocessed_text, df_with_more_features, final_df), axis = 1)

In [41]:
df_complete_test = pd.concat((preprocessed_text_test, df_with_more_features_test, final_df_test), axis = 1)

In [42]:
df_complete.head()

,0,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,when the young people returned to the ballroom...,14,0,0,0,0,11,11,16.454545,181,5.508287,6.494505,11,997,91,591,0.592778
1,all through dinner time mr fayre wa somewhat s...,24,0,5,2,12,10,12,11.466667,172,5.476744,6.482353,15,942,85,551,0.584926
2,a roger had predicted the snow departed a quic...,17,2,1,0,10,11,14,15.636364,172,5.302326,6.273810,11,912,84,527,0.577851
3,and outside before the palace a great garden w...,23,2,0,0,0,5,13,33.400000,167,5.449102,6.095745,5,910,94,573,0.629670
4,once upon a time there were three bear who liv...,13,10,0,0,0,5,12,30.200000,151,4.788079,5.581081,5,723,74,413,0.571231


In [34]:
df_complete_test.head()

,0,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,okay let s get started we can see the eight pe...,268,0,0,0,0,295,15,19.383051,5718,5.482336,7.040509,295,31348,2592,18249,0.582142


In [43]:
preprocessed_text.head()

0    when the young people returned to the ballroom...
1    all through dinner time mr fayre wa somewhat s...
2    a roger had predicted the snow departed a quic...
3    and outside before the palace a great garden w...
4    once upon a time there were three bear who liv...
dtype: object

In [44]:
vectorizer = TfidfVectorizer()
array_output_values = vectorizer.fit_transform(preprocessed_text)

In [45]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [46]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_train['excerpt'])
df_train_converted = vectorizer.transform(df_train['excerpt'])

In [47]:
df_train_converted

<2834x26833 sparse matrix of type '<class 'numpy.float64'>'
	with 294295 stored elements in Compressed Sparse Row format>

In [48]:
df_train_converted

<2834x26833 sparse matrix of type '<class 'numpy.float64'>'
	with 294295 stored elements in Compressed Sparse Row format>

In [67]:
df_complete.head()

,0,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,when the young people returned to the ballroom...,14,0,0,0,0,11,15,16.454545,181,5.508287,6.494505,11,997,91,591,0.592778
1,all through dinner time mr fayre wa somewhat s...,24,0,5,2,12,10,16,11.466667,172,5.476744,6.482353,15,942,85,551,0.584926
2,a roger had predicted the snow departed a quic...,17,2,1,0,10,11,14,15.636364,172,5.302326,6.273810,11,912,84,527,0.577851
3,and outside before the palace a great garden w...,23,2,0,0,0,5,13,33.400000,167,5.449102,6.095745,5,910,94,573,0.629670
4,once upon a time there were three bear who liv...,13,10,0,0,0,5,12,30.200000,151,4.788079,5.581081,5,723,74,413,0.571231


In [41]:
df_complete_test.head()

,0,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,okay let s get started we can see the eight pe...,268,0,0,0,0,295,15,19.383051,5718,5.482336,7.040509,295,31348,2592,18249,0.582142


In [49]:
df_complete_important_features = df_complete.iloc[:, 1:]
df_complete_important_features_test = df_complete_test.iloc[:, 1:]

In [44]:
df_complete_important_features.head()

,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,14,0,0,0,0,11,11,16.454545,181,5.508287,6.494505,11,997,91,591,0.592778
1,24,0,5,2,12,10,12,11.466667,172,5.476744,6.482353,15,942,85,551,0.584926
2,17,2,1,0,10,11,14,15.636364,172,5.302326,6.273810,11,912,84,527,0.577851
3,23,2,0,0,0,5,13,33.400000,167,5.449102,6.095745,5,910,94,573,0.629670
4,13,10,0,0,0,5,12,30.200000,151,4.788079,5.581081,5,723,74,413,0.571231


In [50]:
from sklearn.preprocessing import MinMaxScaler

In [51]:
def min_max_scaler(df_train, df_test):
    """
    This function performs the scaling operation by taking into account the train and test set respectively. It is going to 
    convert the values that are present in the data to lie between 0 and 1 respectively.
    """
    scaler = MinMaxScaler()
    scaler.fit(df_train)
    df_scaled = scaler.transform(df_train)
    df_scaled_test = scaler.transform(df_test)
    return df_scaled, df_scaled_test

In [52]:
def Tfidf_Vectorizer(df_train, df_test):
    """
    This function would take the training data and the test data and convert those values which would be in the form of text 
    to tfidf values that could be later used for machine learning analysis.
    """
    vectorizer = TfidfVectorizer()
    df_vectorized_train = vectorizer.fit_transform(df_train)
    df_vectorized_test = vectorizer.transform(df_test)
    return df_vectorized_train, df_vectorized_test

In [53]:
def Count_Vectorizer(df_train, df_test):
    """
    This function would take the training data and the test data and convert those values which would be in the form of text to 
    bag of words representation or count vectorized values which are later used for machine learning prediction.
    """
    vectorizer = CountVectorizer()
    df_vectorized_train = vectorizer.fit_transform(df_train)
    df_vectorized_test = vectorizer.transform(df_test)
    return df_vectorized_train, df_vectorized_test

In [54]:
df_complete_important_features
df_complete_important_features_test


,num_of_commas,num_of_semicolons,num_of_explamations,num_of_questions,num_of_quotes,num_of_periods,longest_word,sentence_length,num_of_words,word_length,lemma_length,num_of_sentences,initial_text_length,num_of_lemmas,preprocessed_essay_length,text_shortage
0,16,0,0,0,0,9,11,29.9,299,5.013378,6.465517,10,1499,116,750,0.500334


In [55]:
df_train_converted

<2834x26833 sparse matrix of type '<class 'numpy.float64'>'
	with 294295 stored elements in Compressed Sparse Row format>

In [56]:
df_scaled, df_scaled_test = min_max_scaler(df_complete_important_features, df_complete_important_features_test)

In [57]:
def Vectorizer_decision(df_train, df_test):
    print("Please enter the vectorizer that you would like to use for your data")
    print("Please select from the following options")
    print("1. Count Vectorizer")
    print("2. Tfidf Vectorizer")
    choice = int(input("Enter 1 or 2:"))
    type(choice)
    if choice != 1 and choice != 2:
        choice = input("Please select only from the listed options")
        vectorized_train = 0
        vectorized_test = 0
    elif choice == 1:
        print("Count Vectorizer Selected")
        vectorized_train, vectorized_test = Count_Vectorizer(df_train.iloc[:, 0], df_test.iloc[:, 0])
    else:
        print("Tfidf Vectorizer Selected")
        vectorized_train, vectorized_test = Tfidf_Vectorizer(df_train.iloc[:, 0], df_test.iloc[:, 0])
    return vectorized_train, vectorized_test

In [58]:
df_vectorized, df_vectorized_test = Vectorizer_decision(df_complete.copy(), df_complete_test.copy())

Please enter the vectorizer that you would like to use for your data
Please select from the following options
1. Count Vectorizer
2. Tfidf Vectorizer


In [1]:
df_vectorized = df_vectorized.toarray()
df_vectorized_test = df_vectorized_test.toarray()

NameError: name 'df_vectorized' is not defined

In [ ]:
X = np.concatenate((df_vectorized, df_scaled), axis = 1)
X_test = np.concatenate((df_vectorized_test, df_scaled_test), axis = 1)
y = df_train['target'].values

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.3, random_state = 50)

## gradient boosting decision tree for the prediction of the difficulty of the text respectively

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_predict = model.predict(X_cv)

In [ ]:
print("The mean squared error of Gradient Boosting Regressor for cross-validation data is {}".format(mean_squared_error(y_predict, y_cv)))
print("The mean absolute error of Gradient Boosting Regressor for cross-validation data is {}".format(mean_absolute_error(y_predict, y_cv)))

In [ ]:
first_dataframe = pd.DataFrame(pd.Series(y_cv), columns = ['y_actual'])
second_dataframe = pd.DataFrame(pd.Series(y_predict), columns = ['y_predictions'])
final_dataframe = pd.concat((first_dataframe, second_dataframe), axis = 1)

In [ ]:
final_dataframe.head()

In [ ]:
regression_plot_function(final_dataframe, color = 'maroon')